In [43]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import integrate
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import csv, json

In [44]:
ls ..\RawData

 Volume in drive C has no label.
 Volume Serial Number is F6D7-DE80

 Directory of c:\Users\eray2\Documents\Github\semester3\Energy-Consumption-2025\Course code\Assignment 2\RawData

01-11-2025  18:51    <DIR>          .
01-11-2025  18:35    <DIR>          ..
01-11-2025  18:51    <DIR>          Test
               0 File(s)              0 bytes
               3 Dir(s)  2.100.630.556.672 bytes free


In [45]:
input_directory = '..\\RawData'
output_directory = '..\\ChunkedData\\'

In [46]:
os.listdir(input_directory)
onlydir = [f for f in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, f))]
onlyfiles = [f for f in os.listdir(input_directory) if os.path.isfile(os.path.join(input_directory, f))]

In [47]:
onlydir

['Test']

In [48]:
def parse_csv_line(line: str):
    # csv.reader handles the quoted JSON payload with doubled quotes
    return next(csv.reader([line], delimiter=',', quotechar='"', doublequote=True))

In [49]:
def is_json(perhaps_json):
  try:
    json.loads(perhaps_json)
  except ValueError as e:
    return False
  return True


In [51]:
dfs_list = []

dirs = [f for f in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, f))]

for directory in dirs:
    for input_file in os.listdir(input_directory+'/'+directory):
        file_path = input_directory+'/'+directory+'/'
        file_name = os.path.basename(input_file)
        save_lines = False
        if file_name.endswith('.csv'):
            with open(file_path+file_name) as f:
                for line in f:
                # Do something with 'line'        
                #line = msg_line
                    
                    row =  parse_csv_line(line)
                    if row[2] == 'MESSAGE': 
                        row_dict = json.loads(row[3])
                        if is_json(row_dict['message']):
                            message = json.load(row_dict['message'])
                        else:
                            message_list = row_dict['message'].split(',')
                            message = {}
                            # sh post_to_sigless.sh 192.168.50.101:8000 CH1 "stop,<computerID>,<class>"
                            message['action'] = message_list[0]
                            message['computerID'] = message_list[1]
                            message['class'] = message_list[2]
                            
                        print(message)
                        
                        if message['action'] =='start':
                            start_time = row_dict['timestamp']
                            message_values = list(message.values())+[str(start_time)]
                            file_addition = '_'.join(message_values[1:])
                            #print(file_addition)
                            outputfile = output_directory+file_addition+'_'+file_name
                            # Add information from message
                            message.pop('action')
                            start_row = message
                            start_row['start_time'] = start_time
                            start_row['channelId'] =  row_dict['channelId']
                            
                            if(not(save_lines)):
                                f_out = open(outputfile, "w")   # 'r' for reading and 'w' for writing                       
                                save_lines = True
                            else:
                                print(outputfile)
                                f_out = open(outputfile, "w")
            
                        elif message['action'] == 'stop':
                            end_time = row_dict['timestamp']
                            duration = end_time - start_time
                            start_row['exp_duration'] = duration
                            print('meta_row=',start_row)
                            save_lines = False # No more lines for this flow  
                            f_out.close()   # Close output file
                            # save the meta data as the first two lines of the chunked CSV file
                            a = list(start_row.keys())
                            line1 = ','.join(a) 
                            a = list(start_row.values())
                            line2 = ','.join([str(s) for s in a])
                            with open(outputfile, 'r') as original:
                                data = original.read()
                            with open(outputfile, 'w') as modified:
                                modified.write(line1+"\n"+line2+"\n" + data)
                            
                                
                    elif row[2] == 'POWER' and save_lines:  
                        f_out.write(line)   # Write line into the file 



{'action': 'start', 'computerID': 'DK1081104', 'class': 'cpu'}
{'action': 'stop', 'computerID': 'DK1081104', 'class': 'cpu'}
meta_row= {'computerID': 'DK1081104', 'class': 'cpu', 'start_time': 1761131594049, 'channelId': 'CH1', 'exp_duration': 20327}
{'action': 'start', 'computerID': 'DK1081104', 'class': 'cpu'}
{'action': 'stop', 'computerID': 'DK1081104', 'class': 'cpu'}
meta_row= {'computerID': 'DK1081104', 'class': 'cpu', 'start_time': 1761131619395, 'channelId': 'CH1', 'exp_duration': 20377}
{'action': 'start', 'computerID': 'DK1081104', 'class': 'cpu'}
{'action': 'stop', 'computerID': 'DK1081104', 'class': 'cpu'}
meta_row= {'computerID': 'DK1081104', 'class': 'cpu', 'start_time': 1761131644790, 'channelId': 'CH1', 'exp_duration': 20250}
{'action': 'start', 'computerID': 'DK1081104', 'class': 'cpu'}
{'action': 'stop', 'computerID': 'DK1081104', 'class': 'cpu'}
meta_row= {'computerID': 'DK1081104', 'class': 'cpu', 'start_time': 1761131670058, 'channelId': 'CH1', 'exp_duration': 203